# Assignment 3 - Building a Custom Visualization

---

In this assignment you must choose one of the options presented below and submit a visual as well as your source code for peer grading. The details of how you solve the assignment are up to you, although your assignment must use matplotlib so that your peers can evaluate your work. The options differ in challenge level, but there are no grades associated with the challenge level you chose. However, your peers will be asked to ensure you at least met a minimum quality for a given technique in order to pass. Implement the technique fully (or exceed it!) and you should be able to earn full grades for the assignment.


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Ferreira, N., Fisher, D., & Konig, A. C. (2014, April). [Sample-oriented task-driven visualizations: allowing users to make better, more confident decisions.](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;In Proceedings of the SIGCHI Conference on Human Factors in Computing Systems (pp. 571-580). ACM. ([video](https://www.youtube.com/watch?v=BI7GAs-va-Q))


In this [paper](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) the authors describe the challenges users face when trying to make judgements about probabilistic data generated through samples. As an example, they look at a bar chart of four years of data (replicated below in Figure 1). Each year has a y-axis value, which is derived from a sample of a larger dataset. For instance, the first value might be the number votes in a given district or riding for 1992, with the average being around 33,000. On top of this is plotted the 95% confidence interval for the mean (see the boxplot lectures for more information, and the yerr parameter of barcharts).

<br>
<img src="readonly/Assignment3Fig1.png" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Figure 1 from (Ferreira et al, 2014).</h4>

<br>

A challenge that users face is that, for a given y-axis value (e.g. 42,000), it is difficult to know which x-axis values are most likely to be representative, because the confidence levels overlap and their distributions are different (the lengths of the confidence interval bars are unequal). One of the solutions the authors propose for this problem (Figure 2c) is to allow users to indicate the y-axis value of interest (e.g. 42,000) and then draw a horizontal line and color bars based on this value. So bars might be colored red if they are definitely above this value (given the confidence interval), blue if they are definitely below this value, or white if they contain this value.


<br>
<img src="readonly/Assignment3Fig2c.png" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  Figure 2c from (Ferreira et al. 2014). Note that the colorbar legend at the bottom as well as the arrows are not required in the assignment descriptions below.</h4>

<br>
<br>

**Easiest option:** Implement the bar coloring as described above - a color scale with only three colors, (e.g. blue, white, and red). Assume the user provides the y axis value of interest as a parameter or variable.


**Harder option:** Implement the bar coloring as described in the paper, where the color of the bar is actually based on the amount of data covered (e.g. a gradient ranging from dark blue for the distribution being certainly below this y-axis, to white if the value is certainly contained, to dark red if the value is certainly not contained as the distribution is above the axis).

**Even Harder option:** Add interactivity to the above, which allows the user to click on the y axis to set the value of interest. The bar colors should change with respect to what value the user has selected.

**Hardest option:** Allow the user to interactively set a range of y values they are interested in, and recolor based on this (e.g. a y-axis band, see the paper for more details).

---

*Note: The data given for this assignment is not the same as the data used in the article and as a result the visualizations may look a little different.*

In [7]:
%matplotlib widget
"""
Implement the bar coloring as described above - a color scale with only three colors, 
(e.g. blue, white, and red). Assume the user provides the y axis value of interest as a parameter 
or variable.
"""


# Use the following data for this assignment:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

np.random.seed(12345)

df = pd.DataFrame([np.random.normal(32000,200000,3650), 
                   np.random.normal(43000,100000,3650), 
                   np.random.normal(43500,140000,3650), 
                   np.random.normal(48000,70000,3650)], 
                  index=[1992,1993,1994,1995])
df = df.T
df

,1992,1993,1994,1995
0,-8941.531897,-51896.094813,152336.932066,-69708.439062
1,127788.667612,198350.518755,192947.128056,-13289.977022
2,-71887.743011,-123518.252821,389950.263156,-30178.390991
3,-79146.060869,-129916.759685,-93006.152024,55052.181256
4,425156.114501,216119.147314,100818.575896,152883.621657
...,...,...,...,...
3645,24185.008589,-37333.493572,-71861.846997,39937.199964
3646,-56826.729535,103019.841174,26375.113219,139472.114293
3647,-67319.766489,179746.127403,-29328.078384,59386.186379
3648,113377.299342,13455.493990,65858.761714,73362.229590


In [8]:
y_value = 42000
# blue, white, and red
# blue --> value below confidence interval
# white --> value inside confidence interval
# red --> value above confidence interval

# calculate mean, std and count for each year
stats = df.describe().T[["count", "mean", "std"]]
ci95_hi = []
ci95_lo = []

# calculate 95% cinfidence interval
for i in stats.index:
    m, c, s = stats.loc[i]
    ci95_hi.append(m + 1.95*s/math.sqrt(c))
    ci95_lo.append(m - 1.95*s/math.sqrt(c))
    
stats['ci95_lo'] = ci95_lo
stats['ci95_hi'] = ci95_hi

#  define color for each column
stats['colors'] = [*['w'] * len(stats.index)]
# stats['colors'] = stats['ci95_lo'].apply(lambda x: 'b' if x < y_value else np.NaN)
# stats['colors'] = stats['ci95_hi'].apply(lambda x: 'r' if x > y_value else np.NaN)

stats.loc[stats['mean']-stats['ci95_lo'] > y_value, 'colors'] = 'c'
stats.loc[stats['mean']+stats['ci95_hi'] < y_value, 'colors'] = 'r'
stats.loc[(stats['mean']+stats['ci95_hi'] > y_value) & (stats['mean']-stats['ci95_lo'] < y_value), 'colors'] = 'bisque'
stats

,count,mean,std,ci95_lo,ci95_hi,colors
1992,3650.0,33312.107476,200630.901553,1506.461040,5793.538960,r
1993,3650.0,41861.859541,98398.356203,2712.193590,4587.806410,bisque
1994,3650.0,39493.304941,140369.925240,2272.641667,5027.358333,bisque
1995,3650.0,47743.550969,69781.185469,3027.247333,4272.752667,c


In [33]:
labels = list(stats.index.values)
x = np.arange(len(labels))  # the label locations
width = 0.5  # the width of the bars

fig, ax = plt.subplots()
bars = ax.bar(x, stats['mean'], width, yerr=[stats['ci95_lo'], stats['ci95_hi']], capsize=7, color=stats['colors'])

# add horizontal line at y value
line = ax.axhline(y_value, color='g', ls='--')


# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_yticks(list(range(0, 50000, 4000)))
ax.set_yticks(list(range(0, 50000, 4000)))

bars = list(bars)
del bars[1]
legend_labels = ['value above confidence interval', 'value inside confidence interval', 'value below confidence interval','value']
ax.legend([*bars, line], legend_labels, loc='upper center', bbox_to_anchor=(0.5, 1.15), fancybox=True, shadow=True)
ax.text(-0.3, y_value, f'y = {y_value}', color='white', bbox=dict(fc='g',ec='grey', alpha=0.8));
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …